# How to calculate first derivatives

In this guide, we show you how to compute first derivatives with optimagic - while introducing some core concepts.

In [ ]:
import numpy as np
import optimagic as om
import pandas as pd

## Introduction

As in the getting started section, let's look at the sphere function $$f(x) = x^\top x.$$

In [ ]:
def fun_scalar(params):
    return params**2

The derivative of $f$ is given by $f'(x) = 2 x$. With numerical derivatives, we have to
specify the value of $x$ at which we want to compute the derivative. Let's first
consider two **scalar** points $x = 0$ and $x=1$. We have $f'(0) = 0$ and $f'(1) = 2$.

To compute the derivative using optimagic we simply pass the function and the parameters
to the `first_derivative` function:

In [ ]:
fd = om.first_derivative(func=fun_scalar, params=0)
fd.derivative

In [ ]:
fd = om.first_derivative(func=fun_scalar, params=1)
fd.derivative

Notice that the output of `first_derivative` is an object containing the derivative
under the attribute `derivative`. We discuss the ouput in more detail below.

## Gradient and Jacobian

The scalar case from above extends directly to the multivariate case. Let's consider two cases: 

|         |                                     |
|:--------|:------------------------------------|
|Gradient | $f_1: \mathbb{R}^N \to \mathbb{R}$  |
|Jacobian | $f_2: \mathbb{R}^N \to \mathbb{R}^M$|


The first derivative of $f_1$ is usually referred to as the gradient, while the first
derivative of $f_2$ is usually called the Jacobian.

### Gradient

Let's again use the sphere function, but this time with a vector input. The gradient is
a 1-dimensional vector of shape (N,).

In [ ]:
def fun(params):
    return params @ params

In [ ]:
fd = om.first_derivative(fun, params=np.arange(4))
fd.derivative

### Jacobian

As an example, let's now use the function
$$f(x) = (x^\top x) \begin{pmatrix}1\\2\\3 \end{pmatrix},$$
with $f: \mathbb{R}^N \to \mathbb{R}^3$. The Jacobian is a 2-dimensional object of shape
(M, N), where M is the output dimension.

In [ ]:
def fun_multivariate(params):
    return (params @ params) * np.arange(3)

In [ ]:
fd = om.first_derivative(fun_multivariate, params=np.arange(4))
fd.derivative

## The output of ``first_derivative``

The output of `first_derivative` has the following attributes:

- `derivative`: The computed numerical derivative.
- `func_value`: The function value at the params vector, if `return_func_value` is True.

In [ ]:
fd = om.first_derivative(fun_scalar, params=0, return_func_value=True)

In [ ]:
fd.derivative

In [ ]:
assert fd.func_value == fun_scalar(0)

## The `params` argument

Above we used a `float` and `numpy.ndarray` as the `params` argument. In optimagic,
params can be arbitrary [pytrees](https://jax.readthedocs.io/en/latest/pytrees.html).
Examples are (nested) dictionaries of numbers, arrays, and pandas objects. Let's look at
a few cases.

### pandas

In [ ]:
params = pd.DataFrame(
    [["time_pref", "delta", 0.9], ["time_pref", "beta", 0.6], ["price", "price", 2]],
    columns=["category", "name", "value"],
).set_index(["category", "name"])

params

In [ ]:
def fun_pandas(params):
    return params["value"] @ params["value"]

In [ ]:
fd = om.first_derivative(fun_pandas, params)
fd.derivative

### nested dicts

In [ ]:
params = {"a": 0, "b": 1, "c": pd.Series([2, 3, 4])}

params

In [ ]:
def fun_dict(params):
    return params["a"] ** 2 + params["b"] ** 2 + (params["c"] ** 2).sum()

In [ ]:
fd = om.first_derivative(
    func=fun_dict,
    params=params,
)

fd.derivative

### Description of the output

> Note. Understanding the output of the first and second derivative requires terminolgy
> of pytrees. Please refer to the
> [JAX documentation of pytrees](https://jax.readthedocs.io/en/latest/pytrees.html).

The output tree of `first_derivative` has the same structure as the params tree.
Equivalent to the numpy case, where the gradient is a vector of shape `(len(params),)`.
If, however, the params tree contains non-scalar entries, like `numpy.ndarray`'s,
`pandas.Series`', or `pandas.DataFrame`'s, the output is not expanded but a block is
created instead. In the above example, the entry `params["c"]` is a `pandas.Series` with
3 entries. Thus, the first derivative output contains the corresponding 3x1-block of the
gradient at the position `["c"]`:

## Parallelization

Function evaluations can be run in parallel by  by setting the `n_cores` argument. For
example, if we wish to evaluate the function on `2` cores, we simply write

In [ ]:
fd = om.first_derivative(fun_scalar, params=0, n_cores=2)

fd.derivative